<div style="background-color: #333; padding: 40px; border: 2px solid #ffd700; border-radius: 10px; color: #ffd700; text-align: center; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);">

<h1 style="font-size: 48px; font-weight: bold; color: #ffd700;">Superstore Sales</h1>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/12/Walmart_-_Blaine%2C_MN_-_panoramio.jpg/1200px-Walmart_-_Blaine%2C_MN_-_panoramio.jpg" alt="Movie Reel" style="width: 500px; margin: 20px auto; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);">
    
</div>

<div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 180%; text-align: center; color: #ffd700; font-weight: bold;"> Table of Contents 
</div>

<ul class="list-group" id="list-tab" role="tablist">
    <li><a href="#0.-Introduction">0. Introduction</a></li><br>
    <li><a href="#1.-Import-Libraries">1. Import Libraries</a></li><br>
    <li><a href="#2.-Load-data">2. Load data</a></li><br>
    <li><a href="#3.-Exploratory-Data-Analysis">3. Exploratory Data Analysis</a></li><br>
    <ul>
        <li><a href="#3.1-Data-quality">3.1 Data quality</a></li><br>
        <li><a href="#3.2-Time-Series-Analysis">3.2 Time Series Analysis</a></li><br>
        <li><a href="#3.3-Products-Analysis">3.3 Products Analysis</a></li><br>
        <li><a href="#3.4-Geospatial-Analysis">3.4 Geospatial Analysis</a></li><br>
        <li><a href="#3.5-Customers-Segment-Analysis">3.5 Customers Segment Analysis</a></li><br>
        <li><a href="#3.6-Shipping-Mode-Analysis">3.6 Shipping Mode Analysis</a></li><br>
    </ul>
    <li><a href="#4.-Modeling">4. Modeling</a></li><br>
</ul>

## <div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">0. Introduction</div>

# Introduction
The Superstore Sales dataset provides a comprehensive view of sales data over a period, encompassing various product categories, geographic regions, and customer segments. This analysis aims to extract insights and build predictive models to better understand sales trends and make informed business decisions.


# Dataset Overview
The dataset comprises sales data from a superstore, including information such as order date, ship date, product category, sales region, customer segment, shipping mode, and sales amount. It contains both numeric and categorical features, providing a rich source of information for analysis.


# Goal of Analysis
The primary goal is to gain actionable insights into sales trends, patterns, and drivers within the superstore. By exploring the data, we aim to identify key factors influencing sales performance, such as product categories, geographic regions, customer segments, and shipping modes. Additionally, we seek to develop predictive models to forecast future sales accurately.


# Importance of Insights
Understanding sales dynamics is crucial for optimizing inventory management, resource allocation, and marketing strategies. By uncovering insights from the data, the superstore can identify high-performing product categories, target lucrative customer segments, optimize shipping operations, and anticipate demand fluctuations. These insights enable data-driven decision-making, leading to improved operational efficiency, customer satisfaction, and overall business performance.

## <div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">1. Import Libraries</div>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import scipy.stats as stats

warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy")

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer

## <div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">2. Load data</div>

In [ ]:
df = pd.read_csv('/kaggle/input/superstore-sales/superstore_final_dataset (1).csv', encoding='ISO-8859-1', index_col='Row_ID').drop_duplicates()
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

## <div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">3. Exploratory Data Analysis</div>

## <div style="border-radius: 10px; border: 2px solid #333; padding: 15px; background-color: #ffd700; font-size: 120%; text-align: left; color: #333; font-weight: bold;">3.1 Data quality</div>

### I | Check duplicates

In [ ]:
duplicated = df.duplicated().sum()
print(duplicated)

### II | Check null and missing value

In [ ]:
missing_values = df.isnull().sum()
total_missing_values = (missing_values).sum()
total_cells = np.product(df.shape)
percent_missing = (total_missing_values/total_cells) * 100
print("Percent of data that is missing :", percent_missing)
print(missing_values)

In [ ]:
df.dropna(inplace=True)

### III | Check unique values in each columns

In [ ]:
for column in df.columns:
    num_distinct_values = len(df[column].unique())
    print(f"{column}: {num_distinct_values} distinct values")

### IV | Data Type Conversion

In [ ]:
df['Sales'] = df['Sales'].astype(int)

# Change the type of 'Order_Date' and 'Ship_Date'
df['Order_Date'] = pd.to_datetime(df['Order_Date'])
df['Ship_Date'] = pd.to_datetime(df['Ship_Date'])

df['year_order_date'] = df['Order_Date'].dt.year
df['month_order_date'] = df['Order_Date'].dt.month
df['weekday_order_date'] = df['Order_Date'].dt.weekday

df['year_ship_date'] = df['Ship_Date'].dt.year
df['month_ship_date'] = df['Ship_Date'].dt.month
df['weekday_ship_date'] = df['Ship_Date'].dt.weekday

### IV | Correlation Analysis

In [ ]:
numeric_columns = df.select_dtypes(include=[np.number])
correlation_matrix = numeric_columns.corr()
correlation_matrix

In [ ]:
fig, ax = plt.subplots() 
fig.set_size_inches(12,8)
sns.heatmap(correlation_matrix, vmax =.8, square = True, annot = True,cmap='YlGn' )
plt.title('Correlation Matrix',fontsize=15);

In [ ]:
plt.figure(figsize=(16,8))
sns.histplot(data=np.log1p(df['Sales']), bins=40, kde=True)
plt.title("Distribution of sales")
plt.xlabel("Sales")
plt.ylabel("Frequency")
plt.show()

## <div style="border-radius: 10px; border: 2px solid #333; padding: 15px; background-color: #ffd700; font-size: 120%; text-align: left; color: #333; font-weight: bold;">3.2 Time Series Analysis</div>

In [ ]:
# Aggregate sales on a monthly basis for each year
monthly_sales_year = df.resample('M', on='Order_Date').sum()['Sales'].reset_index()

# Create a column for the year
monthly_sales_year['year'] = monthly_sales_year['Order_Date'].dt.year

plt.figure(figsize=(15, 8))
for year in sorted(monthly_sales_year['year'].unique()):
    year_data = monthly_sales_year[monthly_sales_year['year'] == year]
    sns.lineplot(x=year_data['Order_Date'].dt.month, y=year_data['Sales'], label=year, linewidth=0.7)
plt.title('Monthly Sales for Each Year')
plt.xlabel('Month')
plt.ylabel('Sales')
plt.legend(title='Year')
plt.show()

In [ ]:
daily_sales = df.resample('D', on='Order_Date').sum()['Sales']

daily_sales_rolling_7d = daily_sales.rolling(window=7).mean()
daily_sales_rolling_30d = daily_sales.rolling(window=30).mean()

plt.figure(figsize=(15, 6))
sns.lineplot(x=daily_sales.index, y=daily_sales.values, label='Original')
sns.lineplot(x=daily_sales_rolling_7d.index, y=daily_sales_rolling_7d.values, label='7-day Rolling Mean')
sns.lineplot(x=daily_sales_rolling_30d.index, y=daily_sales_rolling_30d.values, label='30-day Rolling Mean')
plt.title('Trend of Sales Over Time with Rolling Means')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))

plt.subplot(1, 2, 1)
sns.kdeplot(data=df, x="Order_Date")
plt.title("Order date density")
plt.xlabel("Order date")

plt.subplot(1, 2, 2)
sns.kdeplot(data=df, x="Ship_Date", color = 'green')
plt.title("Ship date density")
plt.xlabel("Ship date")

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))

plt.subplot(1, 2, 1)
sns.kdeplot(data=df, x="month_order_date")
plt.title("Month order date density")
plt.xlabel("Month Order date")

plt.subplot(1, 2, 2)
sns.kdeplot(data=df, x="month_ship_date", color = 'green')
plt.title("Month Ship date density")
plt.xlabel("Month Ship date")


plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12,6))

plt.subplot(1, 2, 1)
sns.kdeplot(data=df, x="weekday_order_date")
plt.title("Weekday order date density")
plt.xlabel("Weekday Order date")


plt.subplot(1, 2, 2)
sns.kdeplot(data=df, x="weekday_ship_date", color = 'green')
plt.title("Weekday Ship date density")
plt.xlabel("Weekday Ship date")


plt.tight_layout()
plt.show()

## <div style="border-radius: 10px; border: 2px solid #333; padding: 15px; background-color: #ffd700; font-size: 120%; text-align: left; color: #333; font-weight: bold;">3.3 Products Analysis</div>

In [ ]:
plt.figure(figsize=(20,6))
sns.barplot(data=df , x='Category', y = 'Sales')
plt.title("Number of sales per product category")
plt.xlabel("Category")
plt.ylabel("Sales")
plt.show()

In [ ]:
top_10_product = df.groupby('Product_Name')['Sales'].sum().nlargest(10)

plt.figure(figsize=(16,6))
sns.barplot(x=top_10_product.index, y=top_10_product.values)
plt.xlabel('Products')
plt.ylabel('Sales')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
unique_sales_per_subcategory = df.groupby('Sub_Category')['Sales'].sum()

top_10_sub_category = unique_sales_per_subcategory.nlargest(10)

plt.figure(figsize=(16, 6))
sns.barplot(x=top_10_sub_category.index, y=top_10_sub_category.values)
plt.show()

## <div style="border-radius: 10px; border: 2px solid #333; padding: 15px; background-color: #ffd700; font-size: 120%; text-align: left; color: #333; font-weight: bold;">3.4 Geospatial Analysis</div>

In [ ]:
sales_region = ['Sales', 'Region', 'City']
df_sales_region = df[sales_region]


df_sales_region['Sales'] = np.log1p(df_sales_region['Sales'])
top10_sold_Cities = df.groupby('City')['Sales'].sum().nlargest(15)

plt.figure(figsize=(12,6))

plt.subplot(1, 2, 1)
sns.kdeplot(data=df_sales_region, x='Sales', hue='Region')

plt.subplot(1, 2, 2)
sns.barplot(x=top10_sold_Cities.values , y=top10_sold_Cities.index)

plt.tight_layout()
plt.show()

In [ ]:
# Aggregate sales on a monthly basis for each region
monthly_sales_region = df.groupby([df['Order_Date'].dt.to_period('M'), 'Region']).sum()['Sales'].reset_index()
monthly_sales_region['Order_Date'] = monthly_sales_region['Order_Date'].dt.to_timestamp()

# Create a line plot for each region
plt.figure(figsize=(15, 8))
sns.lineplot(data=monthly_sales_region, x='Order_Date', y='Sales', hue='Region')
plt.title('Trend of Sales Over Time for Each Region')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()

## <div style="border-radius: 10px; border: 2px solid #333; padding: 15px; background-color: #ffd700; font-size: 120%; text-align: left; color: #333; font-weight: bold;">3.5 Customers Segment Analysis</div>

In [ ]:
sales_segments = ['Sales', 'Segment']
df_sales_segments = df[sales_segments]

df_sales_segments['Sales'] = np.log1p(df_sales_segments['Sales'])


plt.figure(figsize=(16,6))

plt.subplot(1, 2, 1)
sns.histplot(data=df_sales_segments, x='Sales', hue='Segment')
plt.title("Distributions of Sales per Segment")

plt.subplot(1, 2 ,2)
sns.barplot(data=df_sales_segments, x = 'Segment', y='Sales')
plt.title("Sales vs Segment")
 
plt.tight_layout()
plt.show()

## <div style="border-radius: 10px; border: 2px solid #333; padding: 15px; background-color: #ffd700; font-size: 120%; text-align: left; color: #333; font-weight: bold;">3.6 Shipping Mode Analysis</div>

In [ ]:
plt.figure(figsize=(12,6))

plt.subplot(1, 2, 1)
sns.countplot(data=df, x='Ship_Mode')
plt.title('Distribution of ship mode')
plt.xlabel('Ship mode')

plt.subplot(1, 2, 2)
sns.barplot(data=df, x='Ship_Mode', y='Sales')
plt.title('Ship mode vs Sales')
plt.xlabel('Ship mode')
plt.ylabel('Sales')

plt.tight_layout()
plt.show()

We can see that some shipping modes are more commonly used but generate a lower total sales amount compared to another less utilized mode, indicating that several factors may be at play:

* Lower Average Order Value: The more commonly used shipping mode could be associated with lower-priced products or smaller order sizes, leading to a lower total sales amount despite a higher number of orders.

* Shipping Costs: The less utilized shipping mode might entail higher shipping fees, which could result in a higher total sales amount for that mode despite fewer orders.

* Operational Efficiency: The less utilized shipping mode could be more efficient or less costly for the business, allowing it to generate a higher total sales amount despite fewer orders.

* Customer Preferences: Customers may have preferences for certain shipping modes based on factors like speed, reliability, or other considerations, influencing their purchasing decisions and, consequently, the total sales amount for each shipping mode.

In [ ]:
# Grouping the data by shipping mode
grouped_data = [df[df['Ship_Mode'] == mode]['Sales'] for mode in df['Ship_Mode'].unique()]

# Performing ANOVA test
anova_result = stats.f_oneway(*grouped_data)

print("ANOVA F-value:", anova_result.statistic)
print("ANOVA p-value:", anova_result.pvalue)

p-value obtained from the ANOVA test is greater than 0.05, it suggests that there is no significant difference in sales across different shipping modes. In this case, we fail to reject the null hypothesis, which means we don't have enough evidence to conclude that there is a significant difference in sales between different shipping modes.

## <div style="border-radius: 10px; border: 2px solid #ffd700; padding: 15px; background-color: #333; font-size: 120%; text-align: center; color: #ffd700; font-weight: bold;">4. Modeling</div>

Let's create predictive models using various algorithms such as Linear Regression, Decision Tree, and Random Forest to forecast the sales of products in the Superstore. By training these models on historical sales data and evaluating their performance using metrics like RMSE, MAE, and R-squared, we aim to identify the most accurate model for predicting future sales.

In [ ]:
rmse_results = {}
mae_results = {}
r2score_results = {}

X = df.drop(columns=['Sales'])
y = df['Sales']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing steps for numeric and categorical features
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])


models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42)
}

results = {}

for model_name, model in models.items():
    # Create a pipeline
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # Fit the model
    model_pipeline.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model_pipeline.predict(X_test)
    
    # Evaluate the model using MSE, R-squared, MAE
    rmse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Store the results
    rmse_results[model_name] = rmse
    mae_results[model_name] = mae
    r2score_results[model_name] = r2
    
    print("Model :", model_name)
    print(f"RMSE : {rmse:.2f}")
    print(f"MAE : {mae:.2f}")
    print(f"r2 : {r2:.2f}")
    print("--------")

After evaluating the performance of our predictive models, it's evident that the Random Forest model stands out as the top performer for predicting Superstore product sales.

With an impressively low Root Mean Square Error (RMSE) of approximately 274,850, the Random Forest model's predictions deviate less from the actual sales values on average, surpassing both the Linear Regression and Decision Tree models. Moreover, boasting the lowest Mean Absolute Error (MAE) of 161.02, the Random Forest model demonstrates superior predictive accuracy. Its high coefficient of determination (R-squared) of 0.30 further reinforces its effectiveness, indicating that around 30% of the variance in sales data is captured by the model. These results underscore the Random Forest model's suitability for precise sales prediction, offering valuable insights for optimizing inventory management and decision-making processes.